In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import pyvo as vo

Created TAP+ (v1.0) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


Here's some kit for querying Gaia using the ARI interface -- not sure how quickly this gets updated on day 0, fwiw.

In [15]:
res = Simbad.query_object("hd283809")
# print(res.colnames)
# print(res['RA'].data, res['DEC'].data)

coordstr = '%s %s' % (res['RA'].data[0], res['DEC'].data[0])
print(coordstr)
coord = SkyCoord(coordstr, unit=(u.hourangle, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
print(coord)
print(coord.ra.degree, coord.dec.degree)

tap_service = vo.dal.TAPService("http://gaia.ari.uni-heidelberg.de/tap")
#print(tap_service.tables)


04 41 24.7242 +25 54 48.391
<SkyCoord (ICRS): (ra, dec) in deg
    ( 70.3530175,  25.91344194)>
70.35301749999999 25.913441944444443


In [17]:

tap_results = tap_service.search("SELECT TOP 10 * FROM  gaiadr1.gaia_source")

query_str = """SELECT * FROM gaiadr1.gaia_source
               WHERE CONTAINS(POINT('ICRS',ra,dec),
               CIRCLE('ICRS',%s,%s,0.01))=1""" % (coord.ra.degree,
                                                  coord.dec.degree)
print(query_str)
tap_results = tap_service.search(query_str)
#                                    CIRCLE('ICRS',70.3530175,25.91344194,0.01))=1

for tr in tap_results:
    print(tr['phot_g_mean_mag'])
    print(tr['parallax'], tr['parallax_error'])
    print(tr['phot_g_mean_flux'], tr['phot_g_mean_flux_error'])
    # print(tr.keys())
# gaiadr1.gaia_source

# SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',66.76957,26.10453,0.01))=1"


# r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
# r.pprint()

SELECT * FROM gaiadr1.gaia_source
               WHERE CONTAINS(POINT('ICRS',ra,dec),
               CIRCLE('ICRS',70.35301749999999,25.913441944444443,0.01))=1
9.65653244521
2.52290900132 0.406127295103
2224820.85804 2846.59455459


and this is some kit for querying the Mock Gaia DR2

In [14]:
query_str = '''
WITH sample AS (
    SELECT TOP 10000 * FROM gdr2mock.main
    WHERE (distance(ra, dec, 265.48289546, -25.4975738)<2.5) AND (parallax > 5) AND (parallax < 20) )
SELECT ra, dec, parallax, parallax_error, a0, phot_g_mean_mag
FROM sample'''

#query_str = '''SELECT TOP 1000 * FROM gdr2mock.main'''

mockgaia_tap_service = vo.dal.tap.TAPService("http://dc.zah.uni-heidelberg.de/tap")
async_tap_results = mockgaia_tap_service.run_async(query_str)

In [19]:
async_tap_results.table

ra,dec,parallax,parallax_error,a0,phot_g_mean_mag
deg,deg,mas,mas,mag,mag
float64,float64,float32,float32,float32,float32
263.07159797955399,-26.426072976016201,5.78791,1.2517103,0.35203701,19.9951
263.07996690573498,-26.419605452099901,5.1016998,0.29956523,0.397576,18.1213
263.13654117702703,-26.484835402663901,6.2938299,0.090308636,0.32179901,16.0786
263.11732055366701,-26.4567307851156,5.8276501,0.15960823,0.34956801,17.1306
263.17194858175702,-26.455554571228099,7.36203,0.11633389,0.27667901,16.549801
263.50060014917102,-26.487368974780001,11.0615,0.21820544,0.11764,17.6255
263.51883342968398,-26.5015341491616,7.10569,1.0139152,0.183163,19.732401
263.44062784468099,-26.4556499815513,5.38482,0.13045549,0.24143,16.7638
